<h3> For ingestion of batches of data for prediction</h3>

In [1]:
from helper_functions import synthetic_data_model, patient_id_and_date, clean_up
import hopsworks
from dotenv import load_dotenv
import os

2024-07-18 15:51:20.473237: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-18 15:51:20.524846: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 15:51:20.524876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 15:51:20.526257: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-18 15:51:20.533615: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-18 15:51:20.534561: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
model_dir = "synth_data_model"
model_path = os.path.join(model_dir, "synth_data_model.pkl")

model = synthetic_data_model(model_path)

In [3]:
synthetic_feature_batch = model.sample(100)

In [4]:
synthetic_feature_batch = patient_id_and_date(synthetic_feature_batch)
synthetic_feature_batch = clean_up(synthetic_feature_batch)

In [5]:
synthetic_feature_batch = synthetic_feature_batch.drop("diabetes", axis =1)
synthetic_feature_batch.head()

,patient_id,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,datetime
0,0,43,0,0,36.53,7.0,129,2024-07-18 15:51:31.691770
1,1,30,1,0,37.20,5.7,157,2024-07-18 15:51:31.691770
2,2,44,0,0,29.36,4.5,159,2024-07-18 15:51:31.691770
3,3,80,0,1,25.56,6.0,126,2024-07-18 15:51:31.691770
4,4,60,0,0,34.51,5.6,237,2024-07-18 15:51:31.691770


In [6]:
# conditions = (
#     (synthetic_feature_batch['age'] >= 1) & (synthetic_feature_batch['age'] <= 120) &
#     (synthetic_feature_batch['bmi'] >= 10) & (synthetic_feature_batch['bmi'] <= 100) &
#     (synthetic_feature_batch['HbA1c_level'] >= 3) & (synthetic_feature_batch['HbA1c_level'] <= 15) &
#     (synthetic_feature_batch['blood_glucose_level'] >= 50) & (synthetic_feature_batch['blood_glucose_level'] <= 400))

# synthetic_feature_batch[~conditions]

,patient_id,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,datetime


In [7]:
load_dotenv()
hopswork_key = os.getenv("HOPSWORK_KEY")

In [8]:
project = hopsworks.login(api_key_value=hopswork_key)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/112801
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
fg = fs.get_or_create_feature_group(
    name="diabetes_prediction_features",
    version=1,
    primary_key=['patient_id', 'datetime'],  
    event_time='datetime',
    description="Diabetes features whose label are to be predicted"
)

In [10]:
fg.insert(synthetic_feature_batch, write_options={"wait_for_job" : False})

2024-07-18 15:53:58,607 WARNING: FeatureGroupWarning: The ingested dataframe contains upper case letters in feature names: `['HbA1c_level']`. Feature names are sanitized to lower case in the feature store.

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/112801/fs/112720/fg/1014107


Uploading Dataframe: 0.00% |          | Rows 0/100 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: diabetes_prediction_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/112801/jobs/named/diabetes_prediction_features_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fd78854ada0>, None)